<h1> Segmenting and Classifying Atlanta Neighborhoods <h1>

In [4]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize 
import json

<h2>Introduction-The Problem<h2>

In Atlanta, crime is a major issue. With a crime rate of 58 per 1000 residents, Atlanta is in the 2nd percentile for safety in the United States, meaning it is only safer than 2% of cities in the United States. It's violent crime rate is double the national average. I want to see if there's a meaningful correlation between the venues in a neighborhood and the crime that occurs within that neighborhood.(statistics from https://www.neighborhoodscout.com/ga/atlanta/crime.)

<h2>The Data<h2>

I will be using data from two sources: Foursquare and the Atlanta Police Department

<h3>Foursquare Location Data<h3>

I will be making explore calls using the foursquare API to get location data about each neighborhood. Explore calls return information about a specified number of venues around a specified Latitude and Longitude within a specified radius. Explore calls return lots of information about each venue, but we will only be using the venue's category. Below is an example dataframe with 20 venues within 500m of Atlanta itself.

In [7]:
CLIENT_ID = 'Z2BADHUK0WDYPWNCJKJ11VB5EREKVKIKZMD5JR5NO4IB2QV3'
CLIENT_SECRET = 'IZ1T1SGBG3HPOHRG4JE20IDOV1ITJFTCBVLEMAGC1D1HVUDF'
VERSION = '20201127'
LATITUDE = 33.7490
LONGITUDE = -84.3880
RADIUS = 500
LIMIT = 20
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, LATITUDE, LONGITUDE, RADIUS, LIMIT)
results = requests.get(url).json()
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

<ipython-input-7-c6d8ac1c3da2>:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,The Masquerade,Music Venue,33.751720,-84.389739
1,GSU Sports Arena,College Basketball Court,33.751735,-84.386328
2,Georgia Railroad Freight Depot,Event Space,33.751479,-84.388224
3,Willy's Mexicana Grill #22,Mexican Restaurant,33.751293,-84.385337
4,Jamrock Restaurant,Caribbean Restaurant,33.751554,-84.391356


<h3>Atlanta Police Department Data<h3>

I will be using a csv file from the Atlanta Police Department that contains data about crimes in Atlanta from 2009-2019. Each row is a crime and the dataset's columns include a lot of information about each crime, but we will only be using the type of crime and the neighborhood the crime happened in. The columns are called UCR Literal and Neighborhood. The dataset is called COBRA-2009-2019 and can be found at https://www.atlantapd.org/i-want-to/crime-data-downloads. Below is the first 5 rows of the dataset.

In [8]:
crime_df = pd.read_csv('Atlanta_crime.csv')
crime_df.head()

C:\Users\rohan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,UCR #,IBR Code,Neighborhood,NPU,Latitude,Longitude
0,90010930,2009-01-01,2009-01-01,1145,2009-01-01,1148.0,411.0,NaN,NaN,2841 GREENBRIAR PKWY,Day Watch,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68845,-84.49328
1,90011083,2009-01-01,2009-01-01,1330,2009-01-01,1330.0,511.0,NaN,NaN,12 BROAD ST SW,Day Watch,9,LARCENY-NON VEHICLE,630,2303,Downtown,M,33.75320,-84.39201
2,90011208,2009-01-01,2009-01-01,1500,2009-01-01,1520.0,407.0,NaN,NaN,3500 MARTIN L KING JR DR SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Adamsville,H,33.75735,-84.50282
3,90011218,2009-01-01,2009-01-01,1450,2009-01-01,1510.0,210.0,NaN,NaN,3393 PEACHTREE RD NE,Evening Watch,8,LARCENY-NON VEHICLE,630,2303,Lenox,B,33.84676,-84.36212
4,90011289,2009-01-01,2009-01-01,1600,2009-01-01,1700.0,411.0,NaN,NaN,2841 GREENBRIAR PKWY SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68677,-84.49773
